In [ ]:
import numpy as np
from os.path import isfile
from scipy.io import loadmat
from collections import OrderedDict

from config import DATASET
from train_classifiers import train_classifier
from utils import compute_kernel, compute_precrec
from utils import get_labels, _n_classes, _set_sizes

In [ ]:
# EXP_NAME = 'FK'
EXP_NAME = 'imagenet-caffe-alex'
DIR_DATA = './feature_extraction/' + EXP_NAME + '/codes/'
DIR_SAVE = './feature_extraction/' + EXP_NAME + '/compdata/'

In [ ]:
TrainList = loadmat(DIR_DATA + EXP_NAME + '_train_files.mat')
TrainList = TrainList['train_chunks']
TrainList = np.squeeze(TrainList)
TrainList = np.concatenate(TrainList, axis=0)

ValList = loadmat(DIR_DATA + EXP_NAME + '_val_files.mat')
ValList = ValList['val_chunks']
ValList = np.squeeze(ValList)
ValList = np.concatenate(ValList, axis=0)

TestList = loadmat(DIR_DATA + EXP_NAME + '_test_files.mat')
TestList = TestList['test_chunks']
TestList = np.squeeze(TestList)
TestList = np.concatenate(TestList, axis=0)

In [ ]:
DataList = OrderedDict()
DataList['train'] = TrainList
DataList['val'] = ValList
DataList['test'] = TestList

In [ ]:
if isfile(DIR_SAVE + 'Kernel.npy'):
    print('Loading the kernel matrix ...')
    K = np.load(DIR_SAVE + 'Kernel.npy')
    print('Kernel matrix is loaded.')
else:
    K = compute_kernel(DataList)
    np.save(DIR_SAVE + 'Kernel.npy', K)

In [ ]:
def train_one_vs_all(K, train_set, all_epsilon, all_kappa):
    
    n_classes = _n_classes()    
    set_sizes = _set_sizes()

    tr_size = 0
    for ind, data in enumerate(DATASET):
        if data in train_set:
            tr_size += set_sizes[ind]
    
    K_tr = np.zeros((tr_size, tr_size))
    idx = 0 
    for ind1, tr1 in enumerate(DATASET):
        if tr1 not in train_set:
            continue
        idy = 0
        for ind2, tr2 in enumerate(DATASET):
            if tr2 not in train_set:
                continue
            K_tr[idx:set_sizes[ind1]+idx,
                 idy:set_sizes[ind2]+idy] = K[
                sum(set_sizes[:ind1]):sum(set_sizes[:ind1+1]),
                sum(set_sizes[:ind2]):sum(set_sizes[:ind2+1])]
            idy = set_sizes[ind2]            
        idx = set_sizes[ind1]
    
    labels_raw = get_labels(train_set)

    alpha = np.array([train_classifier(K_tr, labels_raw, all_epsilon, all_kappa, nc)
                      for nc in range(n_classes)])

    return alpha

In [ ]:
def compute_score(K, alpha, train_set, test_set):
    
    n_classes = _n_classes()
    set_sizes = _set_sizes()
    
    tr_size = 0
    ts_size = 0
    for ind, data in enumerate(DATASET):
        if data in train_set:
            tr_size += set_sizes[ind]
        if data in test_set:
            ts_size += set_sizes[ind]
            
    K_tr_ts = np.zeros((tr_size, ts_size))
    idx = 0 
    for ind1, tr1 in enumerate(DATASET):
        if tr1 not in train_set:
            continue
        idy = 0
        for ind2, tr2 in enumerate(DATASET):
            if tr2 not in test_set:
                continue
            K_tr_ts[idx:set_sizes[ind1]+idx,
                    idy:set_sizes[ind2]+idy] = K[
                sum(set_sizes[:ind1]):sum(set_sizes[:ind1+1]),
                sum(set_sizes[:ind2]):sum(set_sizes[:ind2+1])]
            idy = set_sizes[ind2]            
        idx = set_sizes[ind1]
        
    scores = np.zeros((ts_size, n_classes))
    for ci in range(n_classes):
        scores[:,ci] = alpha[ci,:].dot(K_tr_ts)

    return scores

In [ ]:
train_set = ['train']
test_set = ['val']
all_epsilon = np.hstack([np.arange(1, 10) * 1e-4,
                         np.arange(1, 10) * 1e-3,
                         np.arange(1, 11) * 1e-2])
all_kappa = [np.inf]
alpha_train = train_one_vs_all(K, train_set, all_epsilon, all_kappa)

In [ ]:
train_set = ['train']
test_set = ['val']
all_epsilon = np.hstack([np.arange(1, 10) * 1e-4,
                         np.arange(1, 10) * 1e-3,
                         np.arange(1, 11) * 1e-2])
all_kappa = [0.1, 0.2, 0.3, 0.4, 0.5, np.inf]
if isfile(DIR_SAVE + 'alpha_train.npy'):
    print('Loading the trained classifiers ...')
    alpha_train = np.load(DIR_SAVE + 'alpha_train.npy')
    print('Classifiers are loaded.')
else:
    alpha_train = train_one_vs_all(K, train_set, all_epsilon, all_kappa)
    np.save(DIR_SAVE + 'alpha_train.npy', alpha_train)

In [ ]:
AP = np.zeros((len(all_kappa), len(all_epsilon), _n_classes()))
for ind_k in range(len(all_kappa)):
    for ind_e in range(len(all_epsilon)):
        scores = compute_score(
            K, alpha_train[:,:,ind_k,ind_e], train_set, test_set)
        labels = get_labels(test_set)
        AP[ind_k,ind_e,:] = compute_precrec(scores, labels)

In [ ]:
mAP = np.mean(AP, axis=2)
mAP
k_ind, e_ind = np.where(mAP == np.max(mAP[:-1,:]))
c_ind,  = np.where(mAP[-1,:] == np.max(mAP[-1,:]))

In [ ]:
train_set = ['train', 'val']
test_set = ['test']
if isfile(DIR_SAVE + 'alpha_rob.npy'):
    print('Loading the robust classifier ...')
    alpha_rob = np.load(DIR_SAVE + 'alpha_rob.npy')
    print('Classifier is loaded.')
else:    
    c_opt = [all_epsilon[c_ind[0]]]    
    alpha_rob = train_one_vs_all(K, train_set, c_opt, [np.inf]).squeeze()
    np.save(DIR_SAVE + 'alpha_rob.npy', alpha_rob)
if isfile(DIR_SAVE + 'alpha_dro.npy'):
    print('Loading the robust classifier ...')
    alpha_dro = np.load(DIR_SAVE + 'alpha_dro.npy')
    print('Classifier is loaded.')
else:
    epsilon_opt = [all_epsilon[e_ind[0]]]
    kappa_opt = [all_kappa[k_ind[0]]]
    alpha_dro = train_one_vs_all(K, train_set, epsilon_opt, kappa_opt).squeeze()
    np.save(DIR_SAVE + 'alpha_dro.npy', alpha_dro)

In [ ]:
scores_dro = compute_score(K, alpha_dro, train_set, test_set) 
AP_dro = compute_precrec(scores_dro, get_labels(test_set))
scores_rob = compute_score(K, alpha_rob, train_set, test_set) 
AP_rob = compute_precrec(scores_rob, get_labels(test_set))

In [ ]:
AP_rob.mean()

In [ ]:
AP_dro.mean()